# T3.2 Analizar e visualizar estatísticas de datos baixados de MongoDB
## Conexión a MongoDB dende Python e análise de datos

En Atlas mete o dataset de exemplo na túa base de datos.

Engade o enderezo IP autorizado a conectar para que poidas conectar á base de datos.

Proba a conectar dende compass coa URL de conexión que che da.

Da BBDD de proba: sample_mfix, na colección movies: https://www.mongodb.com/docs/atlas/sample-data/sample-mflix/

In [ ]:
#!conda install -y pymongo dnspython

Conecta a Atlas dende Python, carga a colección en Pandas. Averigua o modo máis adecuado.

In [46]:
from pymongo import MongoClient
import pandas as pd

#HOST="(...).mongodb.net"
HOST="diego.hdjanr7.mongodb.net"
PORT=27017
USERNAME="xuwira05"
PASSWORD="abc123."

if HOST == 'localhost':
    if not USERNAME:
        cli_mongo = MongoClient(HOST, PORT)
    else:
        cli_mongo = MongoClient(HOST, PORT, USERNAME, PASSWORD)
else:
    cli_mongo = MongoClient(f"mongodb+srv://{USERNAME}:{PASSWORD}@{HOST}/")

movies = cli_mongo['sample_mflix']['movies']
df = pd.DataFrame(movies.find({}))

A) Contar o total de películas.


In [47]:
len(df)

21349

B) Contar o número de películas de cada xénero.

In [51]:
df.groupby('genres')

C) Contar cantas películas hai por ano.

In [ ]:
df.groupby('year').size()

D) Mirar se hai correlación entre imdb e rotten tomatoes.

In [ ]:
for i in df.columns:
    print(df[df[i]=='imdb'])

E) Contar cantas películas ten cada director

F) Contar en cantas películas ten participado cada actor

G) Contar cantas películas hai de cada idioma

H) Cal é a media de premios das películas

J) Amosar as películas que teñen alomenos 3 premios

K) Contar as películas por país